In [ ]:
import json
import os
import re
from datetime import datetime, timedelta

import pandas as pd
import plotly.graph_objects as go
import yfinance as yf
from IPython.display import display, HTML

from transaction_history_processor import PortfolioHistory

In [ ]:
if not os.path.exists('portfolio_history.json'):
    print("portfolio_history.json not found. Generating it now...")
    portfolio_history = PortfolioHistory()
    portfolio_history.process_transaction_history(
        input_file='transaction_history.json',
        save_output=True,
        output_file='portfolio_history.json'
    )
    print("portfolio_history.json has been generated.")

with open('portfolio_history.json', 'r') as f:
    data = json.load(f)
    sectors = data['sectors']
    portfolio_data = data['portfolios']

In [ ]:
def plot_stock_proportions(portfolio_data, date=None, show_values=True, main_threshold=3.0):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return None

    holdings = portfolio_data[date]['holdings']
    labels = list(holdings.keys())
    values = [float(data['market_value']) for data in holdings.values()]

    total = sum(values)
    percentages = [(value / total) * 100 for value in values]

    main_holdings = list(zip(labels, values, percentages))
    main_holdings.sort(key=lambda x: x[2], reverse=True)

    other_holdings = [h for h in main_holdings if h[2] < main_threshold]
    if len(other_holdings) > 1:
        others_value = sum(v for _, v, _ in other_holdings)
        others_percentage = sum(p for _, _, p in other_holdings)
        main_holdings = [h for h in main_holdings if h[2] >=
                         main_threshold] + [("Others", others_value, others_percentage)]

    main_labels, main_values, main_percentages = zip(*main_holdings)

    fig = go.Figure(data=[go.Pie(labels=main_labels,
                    values=main_percentages, textinfo='label+percent')])

    fig.update_layout(
        title=f'Stock Proportions by Value on {date}',
        showlegend=True,
        legend=dict(title="Holdings")
    )

    if show_values:
        fig.update_traces(
            text=[f'${value:,.2f}' for value in main_values], textposition='inside')

    return fig

# Call the function
plot_stock_proportions(portfolio_data, show_values=False)

In [ ]:
# plot_stock_proportions(
#     portfolio_history, date="2023-10-26")

In [ ]:
def plot_sector_proportions(portfolio_data, sectors, date=None):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return None

    holdings = portfolio_data[date]['holdings']
    sector_counts = {}
    for symbol, data in holdings.items():
        sector = sectors.get(symbol, 'ETF')
        if sector not in sector_counts:
            sector_counts[sector] = 0
        sector_counts[sector] += float(data['quantity']) * \
            float(data['market_price'])

    labels = list(sector_counts.keys())
    sizes = list(sector_counts.values())

    fig = go.Figure(
        data=[go.Pie(labels=labels, values=sizes, textinfo='label+percent')])

    fig.update_layout(
        title=f'Sector Proportions on {date}',
        showlegend=True,
        legend=dict(title="Sectors")
    )

    return fig


# Call the function with the loaded data
plot_sector_proportions(portfolio_data, sectors)

In [ ]:
def plot_portfolio_growth(portfolio_data, plot_sp500=True):
    dates = sorted(portfolio_data.keys())
    unrealized_gains_pct = [float(
        portfolio_data[d]['summary']['unrealized_gain_loss_percentage']) for d in dates]
    unrealized_gains = [float(
        portfolio_data[d]['summary']['unrealized_gain_loss']) for d in dates]
    df_plot = pd.DataFrame({
        'Date': pd.to_datetime(dates),
        'Unrealized Gain/Loss (%)': unrealized_gains_pct,
        'Unrealized Gain/Loss ($)': unrealized_gains
    })
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=df_plot['Date'],
        y=df_plot['Unrealized Gain/Loss (%)'],
        fill='tozeroy',
        fillcolor='rgba(255,0,0,0.2)',
        line=dict(color='red', width=2),
        name='Portfolio',
        hovertemplate='%{y:.2f}% ($%{customdata:.2f})<extra></extra>',
        customdata=df_plot['Unrealized Gain/Loss ($)']
    ))


    if plot_sp500:
        sp500_data = fetch_sp500_data(
            df_plot['Date'].min(), df_plot['Date'].max())
        sp500_performance = (
            (sp500_data['Close'] - sp500_data['Close'].iloc[0]) / sp500_data['Close'].iloc[0]) * 100

        fig.add_trace(go.Scatter(
            x=sp500_data.index,
            y=sp500_performance,
            line=dict(color='blue', width=2),
            name='S&P 500'
        ))

    fig.update_layout(
        title='Portfolio Performance vs S&P 500',
        xaxis_title='Date',
        yaxis_title='Performance (%)',
        yaxis_tickformat='.2f',
        hovermode='x unified',
        showlegend=True,
        yaxis=dict(
            ticksuffix='%',
            range=[min(df_plot['Unrealized Gain/Loss (%)'].min(), sp500_performance.min()) - 5,
                   max(df_plot['Unrealized Gain/Loss (%)'].max(), sp500_performance.max()) + 5]
        ),
        xaxis=dict(
            rangeslider=dict(visible=False),
            rangeselector=dict(
                buttons=list([
                    dict(count=5, label="5D", step="day", stepmode="backward"),
                    dict(count=1, label="1M", step="month", stepmode="backward"),
                    dict(count=6, label="6M", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1Y", step="year", stepmode="backward"),
                    dict(step="all")
                ])
            )
        )
    )

    return fig

def fetch_sp500_data(start_date, end_date):
    # Convert end_date to datetime and add one day
    end_date = pd.to_datetime(end_date) + timedelta(days=1)

    # Download S&P 500 data
    sp500 = yf.Ticker("^GSPC")
    sp500_data = sp500.history(start=start_date, end=end_date)

    # Return only the 'Close' prices
    return sp500_data[['Close']]

In [ ]:
plot_portfolio_growth(portfolio_data)

In [ ]:
def process_portfolio_data(portfolio_data, date=None, sort_by='Symbol', ascending=True):
    if date is None:
        date = max(portfolio_data.keys())
    elif date not in portfolio_data:
        raise ValueError(f"No data available for date: {date}")
    data = portfolio_data[date]

    # Process holdings data
    holdings_data = [
        {
            'Symbol': symbol,
            'Quantity': round(float(info['quantity']), 2),
            'Unit Cost': round(float(info['unit_cost']), 2),
            'Market Price': round(float(info['market_price']), 2),
            'Total Cost': round(float(info['total_cost']), 2),
            'Market Value': round(float(info['market_value']), 2),
            'Unrealized G/L': f"${float(info['unrealized_gain_loss']):,.2f} ({float(info['unrealized_gain_loss_percentage']):.2f}%)",
            'Daily Gain': f"${float(info['daily_gain']):,.2f} ({float(info['daily_return'])*100:.2f}%)"
        }
        for symbol, info in data['holdings'].items()
    ]

    df = pd.DataFrame(holdings_data)
    df = df.sort_values(by=sort_by, ascending=ascending)

    # Process summary data
    summary = data['summary']

    # Process closed positions data
    closed_positions = data.get('closed_positions', {})
    closed_positions_data = [
        {
            'Symbol': symbol,
            'Quantity': round(float(position['quantity']), 2),
            'Sell Price': round(float(position['sell_price']), 2),
            'Cost Basis': round(float(position['cost_basis']), 2),
            'Realized Gain': f"${float(position['realized_gain']):,.2f} ({(float(position['realized_gain']) / float(position['cost_basis']) * 100 if float(position['cost_basis']) != 0 else 0):.2f}%)"
        }
        for symbol, positions in closed_positions.items()
        for position in positions
    ]

    df_closed = pd.DataFrame(closed_positions_data)
    df_closed = df_closed.sort_values(by='Symbol', ascending=True)

    return {
        'date': date,
        'holdings': df,
        'summary': summary,
        'closed_positions': df_closed,
        'realized_gains': data.get('realized_gains', 0),
        'realized_gain_percentage': data.get('realized_gain_percentage', 0)
    }

In [ ]:
def display_portfolio_analysis(processed_data):
    date = processed_data['date']
    df = processed_data['holdings']
    summary = processed_data['summary']
    df_closed = processed_data['closed_positions']

    # Display holdings
    display(HTML(f"<h2>Portfolio Status as of {date}</h2>"))
    display(HTML(df.to_html(index=False, classes='dataframe')))

    # Display summary
    print(f"\nPortfolio Summary:")
    print(f"Cash: ${float(summary.get('cash', 0)):,.2f}")
    print(f"Total Market Value: ${
          float(summary.get('total_market_value', 0)):,.2f}")
    print(f"Invested Value: ${float(summary.get('invested_value', 0)):,.2f}")
    print(f"Unrealized Gain/Loss: ${float(summary.get('unrealized_gain_loss', 0))          :,.2f} ({float(summary.get('unrealized_gain_loss_percentage', 0)):.2f}%)")
    print(f"Daily Gain: ${float(summary.get('daily_gain', 0)):,.2f} ({
          float(summary.get('daily_return', 0))*100:.2f}%)")

    # Display closed positions
    display(HTML("<h2>Closed Positions</h2>"))
    display(HTML(df_closed.to_html(index=False, classes='dataframe')))

    # Display total realized gains
    print(f"\nTotal Realized Gains: ${float(processed_data['realized_gains']):,.2f} ({
          float(processed_data['realized_gain_percentage']):.2f}%)")

In [ ]:
processed_data = process_portfolio_data(portfolio_data)
display_portfolio_analysis(processed_data)

In [ ]:
processed_data = process_portfolio_data(portfolio_data, date="2024-04-10")
display_portfolio_analysis(processed_data)

In [ ]:
def read_template(template_path):
    with open(template_path, 'r') as file:
        return file.read()

def generate_html_report():
    dates = sorted(portfolio_data.keys())
    latest_date = dates[-1]

    stock_proportions_plot = plot_stock_proportions(
        portfolio_data, latest_date)
    sector_proportions_plot = plot_sector_proportions(
        portfolio_data, sectors, latest_date)
    portfolio_growth_plot = plot_portfolio_growth(portfolio_data)

    processed_data = process_portfolio_data(portfolio_data, latest_date)

    # Convert DataFrames to HTML strings
    portfolio_table = processed_data['holdings'].to_html(
        index=False, classes='dataframe')
    closed_positions_table = processed_data['closed_positions'].to_html(
        index=False, classes='dataframe')

    # Format portfolio summary
    summary = processed_data['summary']
    format_dict = {
        'date': latest_date,
        'portfolio_table': portfolio_table,
        'closed_positions_table': closed_positions_table,
        'stock_proportions_plot': stock_proportions_plot.to_json(),
        'sector_proportions_plot': sector_proportions_plot.to_json(),
        'portfolio_growth_plot': portfolio_growth_plot.to_json(),
        'date_options': ' '.join(f'<option value="{date}">{date}</option>' for date in dates),
        'cash': f"${float(summary.get('cash', 0)):,.2f}",
        'total_market_value': f"${float(summary.get('total_market_value', 0)):,.2f}",
        'invested_value': f"${float(summary.get('invested_value', 0)):,.2f}",
        'unrealized_gain_loss': f"${float(summary.get('unrealized_gain_loss', 0)):,.2f} ({float(summary.get('unrealized_gain_loss_percentage', 0)):.2f}%)",
        'daily_gain': f"${float(summary.get('daily_gain', 0)):,.2f} ({float(summary.get('daily_return', 0))*100:.2f}%)",
        'total_realized_gains': f"${float(processed_data['realized_gains']):,.2f} ({float(processed_data['realized_gain_percentage']):.2f}%)"
    }

    # Read the HTML template
    template_path = 'reports/templates/portfolio_report_template.html'
    with open(template_path, 'r') as template_file:
        html_template = template_file.read()

    # Read the CSS content
    css_path = 'reports/templates/portfolio_report_styles.css'
    with open(css_path, 'r') as css_file:
        css_content = css_file.read()

    # Escape curly braces in the CSS content
    css_content_escaped = css_content.replace('{', '{{').replace('}', '}}')

    # Replace the CSS placeholder with the escaped CSS content
    html_template = html_template.replace(
        '/* CSS_PLACEHOLDER */', css_content_escaped)

    # Remove whitespace from placeholders
    html_template = re.sub(r'\{\s*(\w+)\s*\}', r'{\1}', html_template)

    # Generate the HTML content
    html_content = html_template.format(**format_dict)

    # Ensure the 'reports' directory exists
    os.makedirs('reports', exist_ok=True)

    # Generate only the 'latest' version
    today_date = datetime.now().strftime("%Y-%m-%d")
    filename = f'portfolio_report_latest_{today_date}.html'
    with open(os.path.join('reports', filename), 'w') as f:
        f.write(html_content)

    print(f"HTML report generated: reports/{filename}")

In [ ]:
generate_html_report()